In [1]:
from meta_info import *
from tools import *
from plot import *
from ntpath import join
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats
import os
import math
from preprocessing import *

In [6]:
def monthly_compose(dataset, method='mean'):
        
        # 根据method选择聚合方法
        if method == 'mean':
            monthly_data = dataset.resample(time='1MS').mean()
        elif method == 'max':
            monthly_data = dataset.resample(time='1MS').max()
        elif method == 'sum':
            monthly_data = dataset.resample(time='1MS').sum()
        else:
            raise ValueError("Unsupported method. Choose from 'mean', 'max', 'sum'.")
        return monthly_data
def deseason_detrend(dataset , outpath):

        # 计算多年平均值
        multiyear_mean = dataset.mean(dim='time')

        # 筛选出多年平均值大于或等于0.2的像元，不满足条件的设置为NaN
        filtered_dataset = dataset.where(multiyear_mean >= 0.2)

        # 第1步: 去季节性
        # 计算每个月份的多年平均值
        monthly_avg = filtered_dataset.groupby('time.month').mean('time')

        # 从每个原始值中减去对应月份的多年平均值
        deseasonalized = filtered_dataset.groupby('time.month') - monthly_avg

        # 第2步: 去趋势
        deseason_detrend_data = xr.full_like(deseasonalized, fill_value=np.nan)
        
        for lat in deseasonalized.lat:
            for lon in deseasonalized.lon:
                # 获取时间编码作为自变量
                time = np.arange(1,len(deseasonalized.time.dt.month)+1)
                y = deseasonalized.sel(lat=lat, lon=lon).ndvi.values

                #只进行非nan去趋势
                valid_indices = ~np.isnan(y)  # 获取y中非NaN值的索引
                time_clean = time[valid_indices]
                y_clean = y[valid_indices]
              
                if  y_clean.size>1:
                    slope, intercept, _, _, _ = stats.linregress(time_clean, y_clean)
                    trend_line = slope * time + intercept
                    deseason_detrend_data.loc[dict(lat=lat, lon=lon)] =xr.DataArray(y - trend_line, dims=['time'])
                    
        deseason_detrend_data.to_netcdf(outpath)  

In [7]:
datadir = 'E:\\PHD_Project\\Data\\GIMMS3g_NDVI'
# 设置Dask以自动分割大块，避免创建大块
dask.config.set({'array.slicing.split_large_chunks': True})
# 使用通配符指定文件路径
fdir= join(datadir,'Clean\\*.nc4')
outdir = join(datadir,'Deseason_detrend_0.08')
outpath = join(outdir,'deseason_detrend_0.08.nc4')
mk_dir(outdir)
# 打开多个文件并自动合并为一个数据集
dataset = xr.open_mfdataset(fdir, concat_dim='time', combine='nested' )
monthly_data = monthly_compose(dataset, method='max')
deseason_detrend(monthly_data, outpath)

d:\Anaconda\envs\biodiversity\Lib\site-packages\xarray\core\indexing.py:1593: PerformanceWarning: Slicing with an out-of-order index is generating 41 times more chunks
  return self.array[key]
d:\Anaconda\envs\biodiversity\Lib\site-packages\dask\array\reductions.py:654: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
d:\Anaconda\envs\biodiversity\Lib\site-packages\dask\array\reductions.py:654: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
d:\Anaconda\envs\biodiversity\Lib\site-packages\dask\array\reductions.py:654: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
d:\Anaconda\envs\biodiversity\Lib\site-packages\dask\array\reductions.py:654: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)
d:\Anaconda\envs\biodiversity\Lib\site-packages\dask\array\reductions.py:654: RuntimeWarning: A

KeyboardInterrupt: 